In [ ]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
from tqdm import tqdm
import os
import traceback

# 📁 Crear carpeta de modelos
os.makedirs("modelos_guardados", exist_ok=True)

# 📄 Cargar dataset
df = pd.read_csv("datasetwg.csv")
df['periodo'] = pd.to_datetime(df['periodo'])

# 🧮 Agregar por producto y mes
df_agg = df.groupby(['product_id', 'periodo'])['tn'].sum().reset_index()

# 🧽 Filtrar productos válidos
ventas_x_prod = df_agg.groupby('product_id')['tn'].apply(lambda x: (x > 0).sum())
meses_totales = df_agg.groupby('product_id')['periodo'].count()

productos_validos = ventas_x_prod[(ventas_x_prod >= 6) & (meses_totales >= 12)].index.tolist()
productos_ventas_bajas = ventas_x_prod[ventas_x_prod < 6].index.tolist()
productos_pocos_meses = meses_totales[meses_totales < 12].index.tolist()

# 🔁 Loop por producto
resultados = []
errores = []
for prod in tqdm(productos_validos):
    df_prod = df_agg[df_agg['product_id'] == prod].copy()
    df_prod = df_prod.rename(columns={'periodo': 'ds', 'tn': 'y'})
    df_prod = df_prod[['ds', 'y']].sort_values('ds')
    df_prod = df_prod.set_index('ds').asfreq('MS').reset_index()

    try:
        model = NeuralProphet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            n_lags=12,
            n_forecasts=2,
            trend_reg=1
        )
        model.fit(df_prod, freq='MS', progress='off')
        model.save(f"modelos_guardados/neuralprophet_{prod}.np")

        try:
            future = model.make_future_dataframe(df_prod, periods=2, n_historic_predictions=False)
            forecast = model.predict(future)
            pred_feb = forecast.iloc[-1]['yhat2']
        except Exception:
            pred_feb = df_prod[df_prod['y'] > 0]['y'].mean()
            errores.append((prod, 'Fallo en predicción: se usó media positiva'))

    except Exception:
        pred_feb = df_prod[df_prod['y'] > 0]['y'].mean()
        errores.append((prod, 'Fallo en entrenamiento: se usó media positiva'))

    resultados.append((prod, pred_feb))

# 💾 Guardar predicciones
df_resultado = pd.DataFrame(resultados, columns=['product_id', 'tn_predicho'])
df_resultado.to_csv("prediccion_feb2020_neuralprophet_CORREGIDO.csv", index=False)

# 💾 Guardar descartes
df_descartes = pd.DataFrame({
    'product_id': list(set(productos_ventas_bajas + productos_pocos_meses)),
    'motivo': ['ventas < 6' if p in productos_ventas_bajas else 'menos de 12 meses' for p in set(productos_ventas_bajas + productos_pocos_meses)]
})
df_descartes.to_csv("productos_descartados_neuralprophet.csv", index=False)

# 💾 Guardar errores en archivo aparte
if errores:
    df_errores = pd.DataFrame(errores, columns=['product_id', 'motivo'])
    df_errores.to_csv("errores_neuralprophet.csv", index=False)

# 📋 Informe
print("\n📋 INFORME NEURALPROPHET (ROBUSTO)")
print("────────────────────────────────────────")
print(f"🔢 Total productos:        {df_agg['product_id'].nunique()}")
print(f"✅ Procesados:             {len(productos_validos)}")
print(f"❌ Descartados:            {len(df_agg['product_id'].unique()) - len(productos_validos)}")
print(f"   └─ Ventas < 6:          {len(productos_ventas_bajas)}")
print(f"   └─ Menos de 12 meses:   {len(productos_pocos_meses)}")
print(f"⚠️  Errores de fallback:    {len(errores)}")
print("🗃️ Archivos generados: predicción + descartes + errores")
print("💾 Modelos guardados en: modelos_guardados/")


c:\Users\waldo\miniconda3\envs\neuralprophet_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
  0%|          | 0/756 [00:00<?, ?it/s]INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - c:\Users\waldo\miniconda3\envs\neuralprophet_env\lib\site-packages\neuralprophet\df_utils.py:464: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=Fa

Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (3) is too small than the required number                     for the learning rate finder (203). The results might not be optimal.
WARNING - (py.warnings._showwarnmsg) - c:\Users\waldo\miniconda3\envs\neuralprophet_env\lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

WARNING - (py.warnings._showwarnmsg) - c:\Users\waldo\miniconda3\envs\neuralprophet_env\lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 7. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

Finding best initial lr: 100%|██████████| 203/203 [00:03<00:00, 59.76it/s]


Training: |          | 0/? [00:27<?, ?it/s, v_num=0, train_loss=0.0255, reg_loss=0.000, MAE=102.0, RMSE=146.0, Loss=0.0252, RegLoss=0.000]

  0%|          | 1/756 [00:31<6:36:44, 31.53s/it]INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.


...

Epoch 201:  43%|████▎     | 201/470 [00:00<00:00, 1262.02it/s] 
Training: |          | 0/? [00:19<?, ?it/s, v_num=380, train_loss=0.000353, reg_loss=0.000, MAE=0.170, RMSE=0.209, Loss=0.000356, RegLoss=0.000]

INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 8
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 470



Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (3) is too small than the required number                     for the learning rate finder (203). The results might not be optimal.
Finding best initial lr: 100%|██████████| 203/203 [00:02<00:00, 80.66it/s]


Epoch 26:   6%|▌         | 26/470 [00:00<00:00, 10997.57it/s]loss=0.274, reg_loss=0.000, MAE=9.170, RMSE=12.20, Loss=0.276, RegLoss=0.000]